# 0.0 Preparation

In [26]:
# Import Pandas 
import pandas as pd

## Set view to two decimal points only 
pd.set_option('float_format', '{:2f}' .format) 

# Import NumPy
import numpy as np

# Import Scipy
import scipy.cluster.hierarchy as shc
from scipy.cluster.hierarchy import dendrogram, linkage


# Import Matplotlib
import matplotlib.pyplot as plt

# Import from statsmodel
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA


# Import from sci-kit learn
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import AgglomerativeClustering, KMeans

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA


# Import Datetime 
from datetime import datetime as dt, timedelta


# 1.0 Data Loading, Transformation

In [5]:
# Initialize dataframe 
months = [
    "01-2022", "02-2022", "03-2022", "04-2022", "05-2022", "06-2022", 
    "07-2022", "08-2022", "09-2022", "10-2022", "11-2022", "12-2022",
    "01-2023", "02-2023", "03-2023", "04-2023", "05-2023", "06-2023", 
    "07-2023", "08-2023", "09-2023", "10-2023", "11-2023", "12-2023",
    "01-2024", "02-2024", "03-2024", "04-2024", "05-2024", "06-2024", 
    "07-2024", "08-2024", "09-2024", "10-2024", "11-2024", "12-2024"
]

tourist_data = [
    35799, 80882, 150597, 186751, 201368, 193650, 183096, 157338, 
    122373, 135252, 139634, 172852, 177860, 160772, 184970, 213736,
    207512, 196960, 207696, 178589, 124491, 139008, 149625, 179200,
    172913, 174364, 191326, 182647, 205246, 183755, 159826, 121975,
    145021, 162451, 162455, 142728
]

df = pd.DataFrame({
    "month-year" : months,
    "tourists": tourist_data  
})

display(
    df.head(5),
    df.tail(5)
)

,month-year,tourists
0,01-2022,35799
1,02-2022,80882
2,03-2022,150597
3,04-2022,186751
4,05-2022,201368


,month-year,tourists
31,08-2024,121975
32,09-2024,145021
33,10-2024,162451
34,11-2024,162455
35,12-2024,142728


In [9]:
final_df = (
    df
    .assign(date = lambda x: pd.to_datetime(x["month-year"]))
    .assign(
        month = lambda x: x["date"].dt.month,
        year = lambda x: x["date"].dt.year
    )
    .drop(columns = ["month-year", "date"])
    [["month", "year", "tourists"]]
)

final_df

/var/folders/9b/wdj2nc2n1xs_4gffmhjwvngw0000gn/T/ipykernel_6042/489265021.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  .assign(date = lambda x: pd.to_datetime(x["month-year"]))


,month,year,tourists
0,1,2022,35799
1,2,2022,80882
2,3,2022,150597
3,4,2022,186751
4,5,2022,201368
5,6,2022,193650
6,7,2022,183096
7,8,2022,157338
8,9,2022,122373
9,10,2022,135252


In [22]:
# Goal: predict values from 2025

model_df = (
    final_df
    .pivot_table(
        values='tourists',  # Pivot to get total sales as values
        index='month',        # Set 'week' as rows
        columns='year',      # Set 'year' as columns
    )
    .reset_index()
)

model_df

year,month,2022,2023,2024
0,1,35799.000000,177860.000000,172913.000000
1,2,80882.000000,160772.000000,174364.000000
2,3,150597.000000,184970.000000,191326.000000
3,4,186751.000000,213736.000000,182647.000000
4,5,201368.000000,207512.000000,205246.000000
5,6,193650.000000,196960.000000,183755.000000
6,7,183096.000000,207696.000000,159826.000000
7,8,157338.000000,178589.000000,121975.000000
8,9,122373.000000,124491.000000,145021.000000
9,10,135252.000000,139008.000000,162451.000000


In [24]:
# Initialize dictionary to store forecasts
forecasts = {}

# Loop through each month and fit an ARIMA model
for i, month in enumerate(model_df["month"]):
    # Get the time series for the current month (2022, 2023, 2024 values)
    ts = model_df.iloc[i, 1:].values  

    # Fit ARIMA model (p, d, q can be tuned)
    model = ARIMA(ts, order=(1, 1, 1))  
    model_fit = model.fit()
    
    # Forecast 1 step ahead (2025 value)
    forecast = model_fit.forecast(steps=1)[0]
    
    # Store the forecasted value
    forecasts[month] = forecast

# Convert forecasts into a DataFrame
forecast_df = (
    pd.DataFrame(
        list(forecasts.items()), 
        columns=["month", "Forecast_2025"]
        )
        .assign(Forecast_2025 = lambda x: x["Forecast_2025"].astype(int))
        )

forecast_df

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likel

,month,Forecast_2025
0,1,146323
1,2,161727
2,3,192503
3,4,193003
4,5,206013
5,6,190545
6,7,167744
7,8,124315
8,9,156059
9,10,175847


In [25]:
(
    pd.merge(
        model_df,
        forecast_df,
        on = "month", 
        how = "left"

    )
)

,month,2022,2023,2024,Forecast_2025
0,1,35799.000000,177860.000000,172913.000000,146323
1,2,80882.000000,160772.000000,174364.000000,161727
2,3,150597.000000,184970.000000,191326.000000,192503
3,4,186751.000000,213736.000000,182647.000000,193003
4,5,201368.000000,207512.000000,205246.000000,206013
5,6,193650.000000,196960.000000,183755.000000,190545
6,7,183096.000000,207696.000000,159826.000000,167744
7,8,157338.000000,178589.000000,121975.000000,124315
8,9,122373.000000,124491.000000,145021.000000,156059
9,10,135252.000000,139008.000000,162451.000000,175847
